In [1]:
print("Current branch name: ")
!git branch --show-current

try:
    from rich import print
except:
    !pip install rich
    from rich import print

try:
    import tensorflow_datasets, tensorflow, matplotlib, numpy, PIL, sklearn
except:
    !pip install tensorflow_datasets tensorflow matplotlib numpy pillow scikit-learn
    import tensorflow_datasets, tensorflow, matplotlib, numpy, PIL, sklearn

try:
    import keras_tuner as kt
except:
    !pip install keras-tuner
    import keras_tuner as kt

try:
    import cv2
except:
    !pip install opencv-python
    import cv2
import os
import pandas as pd
try:
    import torch
    import numpy as np
    import torchvision.transforms
except:
    !pip install torch torchvision
    import torch

Current branch name: 
3D


/Users/soumensardar/miniconda3/envs/tf/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dataset Selection

In [2]:
# DO NOT TOUCH BELOW ==================================
FULL_DATASET = 0
FULL_BINARY_DATASET = 1
MY_DATASET = 2
MY_BIN_DATASET = 3

# DO NOT TOUCH - END ===================================

# select dataset
SELECTED_DATASET = MY_DATASET # CHANGE HERE



# DO NOT TOUCH BELOW ==================================
FULL_DATASET_PATH = r'D:\CGC work\Alziehmer Disease\PhdNotebook\OASIS dataset\OASIS dataset\Data'
FULL_BINARY_DATASET_PATH = r'D:\CGC work\Alziehmer Disease\PhdNotebook\OASIS dataset Binary\OASIS dataset\Data'
MY_DATASET_PATH = r'/Users/soumensardar/Downloads/OASIS/'
MY_DATASET_BINARY_PATH = r'/Users/soumensardar/Downloads/OASIS-binary/'

# while adding more datasets, make sure to add tflog directory
datasetdir, dataset_name =[  (FULL_DATASET_PATH, 'tflogs_3d'),
                             (FULL_BINARY_DATASET_PATH, 'tflogs_binary_3d'),
                             (MY_DATASET_PATH, 'tflogs_my_3d'),
                             (MY_DATASET_BINARY_PATH, 'tflogs_mybin_3d'),
                    ][SELECTED_DATASET]
# DO NOT TOUCH - END ===================================
datasetdir, dataset_name
assert os.path.exists(datasetdir), f"Dataset path is incorrect {datasetdir}"

# Test Image selection

In [3]:
TEST_IMG_PATH, TEST_IMG_LABEL = [
(r'D:\CGC work\Alziehmer Disease\PhdNotebook\OASIS dataset\OASIS dataset\Data\Mild Dementia/OAS1_0028_MR1_mpr-1_127.jpg', 'Mild_Dementia'),
(r'D:\CGC work\Alziehmer Disease\PhdNotebook\OASIS dataset Binary\OASIS dataset\Data\1. Mild Dementia/OAS1_0028_MR1_mpr-1_127.jpg', '1. Mild_Dementia'),
(r'/Users/soumensardar/Downloads/OASIS/Mild Dementia/OAS1_0028_MR1_mpr-1_127.jpg', 'Mild Dementia'),
(r'/Users/soumensardar/Downloads/OASIS-binary/Mild Dementia/OAS1_0028_MR1_mpr-1_127.jpg', '1. Mild Dementia'),
][SELECTED_DATASET]
import os
assert os.path.exists(TEST_IMG_PATH), f"Test image path is incorrect {TEST_IMG_PATH}"
TEST_IMG_PATH, TEST_IMG_LABEL

('/Users/soumensardar/Downloads/OASIS/Mild Dementia/OAS1_0028_MR1_mpr-1_127.jpg',
 'Mild Dementia')

In [4]:
import os, json
def get_experiment_details(dataset, model, ts):
    exp_asset_dir = f"results/{dataset}/{model}/{ts}"
    assert os.path.exists(exp_asset_dir), f"Experiment does not exist '{exp_asset_dir}'"
    checkpoint_model_dir = None
    with open(exp_asset_dir + "/currentepoch.txt") as fp:
        last_epoch = int(fp.read().strip())
        checkpoint_model_dir = exp_asset_dir + "/models/epoch=%02d.h5"%last_epoch
        assert os.path.exists(checkpoint_model_dir), f"Unable to find the last checkpoint file {checkpoint_model_dir}"
    best_model_dir = exp_asset_dir + "/models/best-model.h5"
    if not os.path.exists(best_model_dir):
        best_model_dir = None

    best_model_info = None
    if os.path.exists(exp_asset_dir + "/bestvalue.json"):
        with open(exp_asset_dir + "/bestvalue.json") as fp:
            best_model_info = json.load(fp)
        
    return dict(last_epoch=last_epoch, 
                best_checkpoint=best_model_dir, 
                last_checkpoint=checkpoint_model_dir,
                best_model_info=best_model_info,
                project_dir=exp_asset_dir)

# <font color='red'>Hyperparameters

In [5]:
random_seed=37
resume_training_timestamp = None # CHANGE HERE
max_epoch = 50 # CHANGE HERE
batch_size = 64 # CHANGE HERE
image_size = 128
color_mode = 'rgb'
n_ch = dict(rgb=3, grayscale=1)[color_mode]
monitor = "f1_score"
initial_threshold = 0.5
mode="max"
freq="epoch"
initial_epoch=0

learning_rate = 0.001
lr_schedular = False

enable_class_weight = False
ablation_study_size = 0 # batch_size * 30 # CHANGE HERE

task_type = "categorical" # "binary"
# Link: https://www.tensorflow.org/api_docs/python/tf/keras/losses#functions
if task_type == "binary":
    activation = "sigmoid"
    loss_function = "binary_crossentropy"
else:
    activation = "softmax"
    loss_function = "categorical_crossentropy"

resume_checkpoint_path = None
# Link: https://keras.io/api/applications/
model_name="3DMobileNet" # CHANGE HERE !!!!!!!
# add a comment about what changes you have done just now before running the training
what_changed = f"Training with {model_name=} {task_type=} {lr_schedular=} {activation=} {loss_function=}"
print(f"{what_changed=}")

what_changed="Training with model_name='3DMobileNet' task_type='categorical' lr_schedular=False 
activation='softmax' loss_function='categorical_crossentropy'"

In [6]:
import tensorflow as tf
# EACH LINK CONTAINS AVAILABLE OPTIONS
# Link: https://www.tensorflow.org/api_docs/python/tf/keras/optimizers#classes
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate) # CHANGE HERE
# Link: https://www.tensorflow.org/api_docs/python/tf/keras/metrics#classes
metrics = [tf.keras.metrics.Accuracy(), 
           tf.keras.metrics.Recall(), 
           tf.keras.metrics.Precision(),
           tf.keras.metrics.F1Score(average="macro"),
           tf.keras.metrics.SensitivityAtSpecificity(0.6),
           tf.keras.metrics.SpecificityAtSensitivity(0.6),
          ]

2024-12-21 00:45:38.591455: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-12-21 00:45:38.591477: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-12-21 00:45:38.591483: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-12-21 00:45:38.591672: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-21 00:45:38.591840: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
# Learning rate schedular callback
def lr_schedule(epoch):
  """
  Returns a custom learning rate that decreases as epochs progress.
  """
  learning_rate = 0.02
  if epoch > 5:
    learning_rate = 0.01
  if epoch > 10:
    learning_rate = 0.0001
  if epoch > 15:
    learning_rate = 0.00001

  tf.summary.scalar('learning rate', data=learning_rate, step=epoch)
  return learning_rate

In [8]:
if resume_training_timestamp:
    print(f"Trying to resume from checkpoint... {resume_training_timestamp}")
    d = get_experiment_details(dataset_name, model_name, resume_training_timestamp)
    initial_epoch = d['last_epoch']
    resume_checkpoint_path = d['last_checkpoint']
    assert os.path.exists(resume_checkpoint_path), f"Unable to resume training from '{d['project_dir']}'."
    best_model_info = d['best_model_info']
    if best_model_info:
        print("Updating the metric monitoring parameters before resuming the checkpoint")
        monitor = best_model_info['monitor']
        initial_threshold = best_model_info['value']
        mode=best_model_info['mode']
        freq=best_model_info['frequency']
    print(f"Resuming checkpoint form epoch={initial_epoch}.")
# =====================================================


def save_hparams():
    hyprams = dict(
            timestamp = timestamp,
            what_changed=what_changed,
            random_state=random_seed,
            max_epoch=max_epoch,
            batch_size=batch_size,
            image_size=image_size,
            num_channels=n_ch,
            metrics=str(metrics),
            loss_function=loss_function,
            monitoring=dict(monitor = monitor, initial_threshold = initial_threshold, mode=mode),
            resume_training_timestamp=resume_training_timestamp,
            initial_epoch=initial_epoch,
            model_name=model_name,
            optimizer=optimizer.name,
            learning_rate=learning_rate,
            lr_schedular=lr_schedular,
        )
    import json 
    log("Saving hyperparameters.")
    print(hyprams)
    # Convert and write JSON object to file
    with open(f"{artifact_root}/hyperparams.json", "w") as outfile: 
        json.dump(hyprams, outfile, indent=4)
    file_writer = tf.summary.create_file_writer(tf_log_dir + "/hparams")
    with file_writer.as_default():
        tf.summary.text("hyperparams.json", f"{artifact_root}/hyperparams.json", step=0)
        for k, v in hyprams.items():
            if isinstance(v, int):
                tf.summary.scalar(k, v, step=0)
            elif isinstance(v, float):
                tf.summary.scalar(k, v, step=0)
            else:
                tf.summary.text(k, str(v), step=0)

# Prepare artifact directory

In [9]:
import pathlib
import datetime

timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
if resume_training_timestamp:
    print("Resume timestamp", resume_training_timestamp)
    timestamp = resume_training_timestamp
unique_dir = f"{model_name}/{timestamp}"
tf_log_dir = f"results/{dataset_name}/{unique_dir}"
tf_log_img_dir = f"results/{dataset_name}/images"
artifact_root  = f"results/{dataset_name}/{unique_dir}"
pathlib.Path(artifact_root).mkdir(parents=True, exist_ok=True)
pathlib.Path(tf_log_dir).mkdir(parents=True, exist_ok=True)

def log(*args, **kwargs):
    time = False
    if "time" in kwargs.keys():
        time = kwargs["time"]
        del kwargs["time"]
    if time:
        time = datetime.datetime.now().strftime("%H:%M:%S %d/%m/%Y")
        a = list(args)
        a.append(time)
        args = tuple(a)
    print(*args, **kwargs)
    with open(f"{artifact_root}/additional_logs.txt", "a") as fp:
        kwargs["file"] = fp
        kwargs["flush"] = True
        print(*args, **kwargs)
log(f"Experiment path: '{artifact_root}'")

Experiment path: 'results/tflogs_my_3d/3DMobileNet/20241221-004538'

Experiment path: 'results/tflogs_my_3d/3DMobileNet/20241221-004538'

# <font color='red'>Training Resume Timestamp
## Use this `timestamp` to update `resume_training_timestamp` variable

In [10]:
log(f"Resume timestamp: {timestamp}")

Resume timestamp: 20241221-004538

Resume timestamp: 20241221-004538

In [11]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D
#   DataGenerator to read images and rescale images
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
import os
import random
tf.random.set_seed(random_seed)
random.seed(random_seed)
np.random.seed(random_seed)

# Loading 3D datasets

In [12]:
import os
import pathlib
import random
import warnings

import cv2
import numpy as np
import tensorflow as tf
from torch.utils.data import Dataset


class OASISTorchDataset(Dataset):
    BINARY_TASK = "binary"
    CATEGORY_TASK = "categorical"
    VOLUME_DEPTH = 61

    def __init__(self, path, task_type, image_size, transforms, nch=1, seed=37, class_names=None, dtype="float32"):
        self.path = pathlib.Path(path)
        self.classnames = [i for i in os.listdir(path)]
        assert len(image_size) == 2, "Image size must be a tuple of two integers"
        self.image_size = image_size
        self.dtype = dtype
        self.transforms = transforms
        if class_names is not None:
            self.classnames = [c for c in self.classnames if c in class_names]
        self.num_classes = len(self.classnames)
        if task_type == self.BINARY_TASK:
            self.num_classes = 1
        self.task_type = task_type
        assert nch in (1, 3), "Supported channels are 1(greyscale) and 3(rgb)"
        self.nch = nch
        if self.task_type == self.BINARY_TASK and len(self.classnames) >= 2 and class_names is None:
            raise Exception(f"When `class_names` is not specified, "
                            f"only 2 classes are allowed in the data-directory, but {len(self.classnames)} found.")
        self.items = []
        self._load(seed)

    @staticmethod
    def _group_images_by_subject(image_dir):
        """Groups images by subject ID based on their filenames.
        Args:
            image_dir: Path to the directory containing the images.
        Returns:
            A dictionary where keys are subject IDs and values are lists of image filenames.
        """
        image_files = os.listdir(image_dir)
        subject_groups = {}
        for filename in image_files:
            # Extract subject ID from the filename (adjust the pattern as needed)
            subject_id = "_".join(filename.split('_')[0:-1])
            if subject_id not in subject_groups:
                subject_groups[subject_id] = []
            subject_groups[subject_id].append(os.path.join(image_dir, filename))
        print("For", len(subject_groups.keys()), "patients,", len(image_files), f"images scanned from '{image_dir}'")
        return subject_groups

    def item_generator(self):
        """
        :return: yield MRI slices as list and encoded label
        """
        for class_dir in self.classnames:
            grouped_files = self._group_images_by_subject((self.path / class_dir).resolve())
            for slices, label in zip(grouped_files.values(), [class_dir] * len(grouped_files.values())):
                lbl = self._encode_labels(label)
                yield slices, lbl

    def volume_generator(self):
        """
        :return: yield MRI slices as 3D RGB image volume and encoded label
        """
        for class_dir in self.classnames:
            grouped_files = self._group_images_by_subject((self.path / class_dir).resolve())
            for slices, label in zip(grouped_files.values(), [class_dir] * len(grouped_files.values())):
                vol = self._load_volume(slices)
                lbl = self._encode_labels(label)
                yield vol, lbl

    def _encode_labels(self, labels):
        """
        :param labels: this could be a string or list of strings
        :return: encoded binary tensor of [1,0,1,...] or [[0,1],[1,0],[0,1],...]
                for categorical
        """
        is_one_element = False
        if isinstance(labels, str):
            labels = [labels]
            is_one_element = True
        enc_label = []
        for lbl in labels:
            if lbl not in self.classnames:
                warnings.warn(f"'{lbl}' label is unknown")
            if self.task_type == self.BINARY_TASK:
                enc_label.append(self.classnames.index(lbl))
            else:
                enc_label.append([lbl == cls_nm for cls_nm in self.classnames])
        if is_one_element:
            return np.array(enc_label[0]).astype(self.dtype)
        else:
            return np.array(enc_label).astype(self.dtype)

    def _decode_labels(self, probabilities, probability_threshold=0.5):
        """
        :param probabilities: batch of the network output(probabilities)
        :param probability_threshold: to decide the class
        :return: decode [probas1, probas2, ...] to tensor of [cls1, cls2,...]
        """
        assert len(probabilities.shape) == 2, f"`probabilities` shape must be two dimensional"
        if self.task_type == self.BINARY_TASK:
            assert probabilities.shape[1] == 1, f"`probabilities` shape must be (batch_size, 1)"
            binaries = probabilities >= probability_threshold
            return list(map(lambda oi: self.classnames[int(oi)], binaries))
        else:
            assert probabilities.shape[1] == len(self.classnames), (f"`probabilities` shape must be"
                                                                    f" (batch_size, num_classes)")
            probability_threshold = [probability_threshold] * len(self.classnames)
            indices = (probabilities >= probability_threshold).numpy().argmax(axis=1)
            string_array = np.array(self.classnames)
            return string_array[indices].tolist()

    def _load_volume(self, image_files: list, label=None):
        """Loads a 3D volume from a directory of JPEG images.
        Args:
            image_files: list of files
        Returns:
            A 3D representing the 3D volume
        """
        image_volume = []
        for image_path in image_files:
            image_data = cv2.imread(image_path)
            if self.nch == 1:
                image_data = cv2.cvtColor(image_data, cv2.COLOR_BGR2GRAY)
            else:
                image_data = cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB)
            # resize
            image_data = cv2.resize(image_data, self.image_size)
            # transforming images
            image_data = self.transforms(image_data.astype("uint8"))
            # creating volume
            image_volume.append(image_data)
        volume = np.stack(image_volume, axis=0).astype(self.dtype)
        if label is None:
            return volume
        else:
            return volume, label

    def _load(self, seed):
        random.seed(seed)
        self.items = list(self.item_generator())
        random.shuffle(self.items)
        return self

    def __len__(self, ):
        return len(self.items)

    def __getitem__(self, idx):
        return self._load_volume(*self.items[idx])


def torch_to_tf(torch_dataset, task_type, image_size, nch=3, dtype="float32"):
    def torch_data_gen():
        for img, lbl in torch_dataset:
            yield tf.convert_to_tensor(img), tf.convert_to_tensor(lbl)

    input_shape = (61, *image_size, nch)
    label_shape = (1,) if task_type == OASISTorchDataset.BINARY_TASK else (4,)
    input_datatype = (getattr(tf, dtype), getattr(tf, dtype))
    dataset = tf.data.Dataset.from_generator(torch_data_gen,
                                             output_types=input_datatype,
                                             output_shapes=(input_shape, label_shape)
                                             )
    return dataset


class OASIS_TF:
    def __init__(self):
        self.classnames = None
        self.num_classes = None
        self.num_items = None

    def tf_oasis_load_dataset(
            self,
            directory,
            transforms,
            label_mode="category",
            class_names=None,
            color_mode="rgb",
            batch_size=32,
            image_size=(256, 256),
            seed=None,
            validation_split=0.2,
    ):
        assert color_mode in ("greyscale", "rgb"), color_mode
        assert label_mode in (OASISTorchDataset.BINARY_TASK, OASISTorchDataset.CATEGORY_TASK), label_mode
        nch = 3 if color_mode == "rgb" else 1
        dtype = "float32"
        # load torch dataset
        torch_dataset = OASISTorchDataset(directory,
                                          task_type=label_mode,
                                          image_size=image_size,
                                          transforms=transforms,
                                          nch=nch,
                                          seed=seed,
                                          class_names=class_names,
                                          dtype=dtype)
        self.classnames = torch_dataset.classnames
        self.num_classes = torch_dataset.num_classes
        self.num_items = len(torch_dataset)
        print("Number of patients:", self.num_items)
        print("Class Names:", self.classnames)
        print("Number of classes:", self.num_classes)
        print(f"Minimum data waste for {batch_size=} is", self.num_items % batch_size)
        # convert torch to tf dataset
        dataset = torch_to_tf(torch_dataset,
                              task_type=label_mode,
                              image_size=image_size,
                              nch=nch, dtype="float32")
        # calculate split sizes
        dataset_size = len(torch_dataset)  # Assuming dataset has a defined length
        train_size = int(1. - validation_split * dataset_size)
        # split
        train_dataset = dataset.take(train_size)
        val_dataset = dataset.skip(train_size)
        # batch
        train_dataset = train_dataset.batch(batch_size)
        val_dataset = val_dataset.batch(batch_size)
        return train_dataset, val_dataset

    def __len__(self):
        return self.num_items


## Dataset loading optimization and normalization

In [13]:
def bilateral_filter(pil_image):
    return cv2.bilateralFilter(np.array(pil_image), 15, 75, 75)


transforms = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.RandomVerticalFlip(),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToPILImage(),
        torchvision.transforms.Lambda(bilateral_filter),
        torchvision.transforms.Normalize(mean=0., std=1.),
])

odd = OASIS_TF()

train_val_ds, test_ds = odd.tf_oasis_load_dataset(
    datasetdir,
    transforms=transforms,
    label_mode=task_type,
    color_mode=color_mode,
    batch_size=batch_size,
    image_size=(image_size, image_size),
    seed=random_seed,
    validation_split=0.2,
)

CLASS_NAMES = odd.classnames
num_classes = odd.num_classes

For 82 patients, 5002 images scanned from '/Users/soumensardar/Downloads/OASIS/Mild Dementia'

For 225 patients, 13725 images scanned from '/Users/soumensardar/Downloads/OASIS/Very mild Dementia'

For 8 patients, 488 images scanned from '/Users/soumensardar/Downloads/OASIS/Moderate Dementia'

For 1102 patients, 67222 images scanned from '/Users/soumensardar/Downloads/OASIS/Non Demented'

Number of patients: 1417

Class Names:
['Mild Dementia', 'Very mild Dementia', 'Moderate Dementia', 'Non Demented']

Number of classes: 4

Minimum data waste for batch_size=64 is 9

## Dataset split

In [14]:
print(CLASS_NAMES)

# Reducing dataset size for ablation study
if ablation_study_size > 0:
    batch_count = int(round(ablation_study_size / batch_size, 0))
    train_val_ds = train_val_ds.take(batch_count)

# Create a test dataset by further splitting the validation set
val_batches = tf.data.experimental.cardinality(train_val_ds)

val_ds = train_val_ds.take(val_batches // 5) # 20% of validation as test
train_ds = train_val_ds.skip(val_batches // 5)



train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

['Mild Dementia', 'Very mild Dementia', 'Moderate Dementia', 'Non Demented']

In [15]:
class_weights = None
"class_weights", class_weights, "=============="

('class_weights', None, '==============')

# Logger and Callbacks

In [16]:
# CSV Logger
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping
from tensorflow.keras.callbacks import LambdaCallback, Callback, LearningRateScheduler
import shutil, glob

# Create EarlyStopping callback
early_stopping = EarlyStopping(monitor="val_loss", 
                               mode="min", 
                               patience=7, 
                               min_delta=0.001,
                               verbose=1,
                               start_from_epoch=9,
                               restore_best_weights=True)

csv_logger = CSVLogger(artifact_root + '/metrics.csv')
# Tensorboard Logger
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=tf_log_dir,
    histogram_freq=0,
    write_graph=False,
    write_images=False,
    write_steps_per_second=False,
    update_freq='epoch',
    profile_batch=0,
    embeddings_freq=0,
    embeddings_metadata=None    
)

model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    artifact_root + "/models/epoch={epoch:02d}.h5",
    monitor=monitor,
    verbose=0,
    save_best_only=False,
    save_weights_only=False,
    mode=mode,
    save_freq=freq,
    initial_value_threshold=initial_threshold,
)

model_best_ckpt = tf.keras.callbacks.ModelCheckpoint(
    artifact_root + "/models/best-model.h5",
    monitor=monitor,
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode=mode,
    save_freq=freq,
    initial_value_threshold=initial_threshold,
)

class BestModelEpochTrackerCallback(Callback):
    """
    This callback monitor best values and updates in a json file project_dir/bestvalue.json
    """
    def __init__(self, monitor, mode, initial_value_threshold=None, verbose=0):
        assert mode in ("min", "max")
        initial_thresh = initial_value_threshold
        self.monitor = monitor
        self.mode = mode
        self.best_value = initial_thresh
        if mode == "min":
            self.is_better = np.less
            if self.best_value is None:
                self.best_value = np.Inf
        elif mode == "max":
            self.is_better = np.greater
            if self.best_value is None:
                self.best_value = -np.Inf 
        self.verbose = verbose
        
    def on_epoch_end(self, epoch, metrics=None):
        curr_val = metrics.get(self.monitor, None)
        assert curr_val is not None, f"Unable to find the metric to monitor: {self.monitor}"
        if self.is_better(curr_val, self.best_value):
            update_path = artifact_root + "/bestvalue.json"
            if self.verbose:
                print(f"Epoch {epoch+1}: {self.monitor} improved form {self.best_value:.5f} to {curr_val:.5f} and saving updates to {update_path}")
            self.best_value = curr_val
            with open(update_path, "w") as fp:
                json.dump(dict(epoch=epoch+1, 
                               monitor=self.monitor, 
                               value=curr_val, 
                               mode=self.mode, 
                               frequency="epoch"
                              ), fp, indent=4)
        else:
            if self.verbose:
                self.print(f"Epoch {epoch+1}: {self.monitor} did not improved form {self.best_value}")
            
                

bestval_monitor_callback = BestModelEpochTrackerCallback(
    monitor=monitor,
    mode=mode,
    initial_value_threshold=initial_threshold,
)



class CleanupCallback(Callback):
    def on_epoch_begin(self, epoch, metrics=None):
        # clean last checkpoint assets
        last_epoch = epoch - 1
        # update current
        with open(artifact_root + "/currentepoch.txt", "w") as fp:
            fp.write(f"{epoch}")
        # look for last epoch checkpoint and delete
        pattern = artifact_root + "/models/epoch=%02d.h5" % (last_epoch)
        if os.path.exists(pattern):
            os.remove(pattern)
        
        
cleanup_callback = CleanupCallback()

lr_callback = LearningRateScheduler(lr_schedule)
if lr_schedular:
    callbacks = [lr_callback]
else:
    callbacks = []

callbacks.extend([csv_logger, tensorboard_callback, model_ckpt, model_best_ckpt, bestval_monitor_callback, cleanup_callback])


# Model 3D build

In [17]:
# Define the model architecture (ResNet50)
# import tensorflow.keras as K
log("Building model...")
import keras as K
    
input_t= K.Input(shape=(image_size,image_size,n_ch))
res_model= getattr(K.applications, model_name)(include_top=False, weights="imagenet", input_tensor=input_t)
model_lenet = K.models.Sequential()
model_lenet.add(res_model)
model_lenet.add(K.layers.Flatten())
model_lenet.add(K.layers.Dense(num_classes, activation=activation))


# Compile the model
log("Compile model")
model_lenet.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)

Building model...

Building model...

AttributeError: module 'keras.applications' has no attribute '3DMobileNet'

# Training

In [ ]:
# reload checkpoint
if resume_checkpoint_path and os.path.exists(resume_checkpoint_path):
    log("Resuming training...", resume_checkpoint_path)
    model_lenet.load_weights(resume_checkpoint_path)
else:
    log("Fresh training...")
    initial_epoch = 0
# daving hyper-params
save_hparams()


# Train the model
log("Experiment: Started", time=True)
log(f"Starting training model={model_name}")
history = model_lenet.fit(train_ds, epochs=max_epoch, 
                          initial_epoch=initial_epoch, 
                          batch_size=batch_size, 
                          callbacks = callbacks,
                          class_weight=class_weights,
                          validation_data=val_ds)
log(f"Training done={model_name}")

In [ ]:
# Evaluate the model on the test set
log("Experiment: Evaluating", time=True)
log(f"Evaluating model={model_name}...")
test_result = model_lenet.evaluate(test_ds)

In [ ]:
class LossMetric(tf.keras.metrics.Metric):
    def __init__(self, name='loss', **kwargs):
        super().__init__(name='loss', **kwargs)

    def update_state(self, y_true, y_pred, sample_weight=None):
        pass

    def result(self):
        return 0

In [ ]:
metrics.append(LossMetric())

log(f"{model_name}", {f"test_{k.name}":v for v,k in zip(test_result, metrics)})

# TF-logging test values
file_writer = tf.summary.create_file_writer(tf_log_dir + "/test")
with file_writer.as_default():
    for v, k in zip(test_result, metrics):
        tf.summary.scalar(f"test_{k.name}", v, step=0)

In [ ]:
from PIL import Image
def preprocess_image(image_path, target_size=(image_size, image_size)):
    # Open the image file
    img = Image.open(image_path)
    # Resize the image
    img = img.resize(target_size)
    # Convert the image to a NumPy array
    img_array = np.array(img)
    # Expand dimensions to match the shape expected by the model (if necessary)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array


# Example usage:
actual_class = TEST_IMG_LABEL
image_path = TEST_IMG_PATH
image_array = preprocess_image(image_path)
log("Shape of preprocessed image array:", image_array.shape)

# Predict probabilities
log("Experiment: Testing", time=True)
prediction_probabilities = model_lenet.predict(image_array)

# Get the index of the highest probability
predicted_class_index = np.argmax(prediction_probabilities)

# Define your class labels
class_labels = CLASS_NAMES

# Map the index to the corresponding class label
predicted_class = class_labels[predicted_class_index]

# TF-logging test values
file_writer = tf.summary.create_file_writer(tf_log_dir + "/test")
with file_writer.as_default():
    tf.summary.text(f"Corner Case", f"Actual={actual_class} Predicted={predicted_class}", step=0)

log("Actual class:", actual_class)
log("Predicted class:", predicted_class)
log("Experiment: Completed", time=True)

with file_writer.as_default():
    # Don't forget to reshape.
    figure = plt.figure(figsize=(5,5))
    images = np.reshape(image_array, (image_size, image_size, n_ch))
    plt.imshow(images)
    plt.title(f"Actual={actual_class} vs Predicted={predicted_class}")
    # TODO add actual vs predicted label
    images = plot_to_image(figure)
    tf.summary.image(f"Test Image", images, max_outputs=1, step=0)

In [ ]:
log(f"Your result is saved at: '{artifact_root}'")

In [ ]:
# # start tensorboard
# # Load the TensorBoard notebook extension
# %reload_ext tensorboard
# if SELECTED_DATASET == SMALL_DATASET:
#     %tensorboard --logdir "results/tflogs_smalldataset/" 
# elif SELECTED_DATASET == FULL_DATASET:
#     %tensorboard --logdir "results/tflog/"
# elif SELECTED_DATASET == FLOWER_DATASET:
#     %tensorboard --logdir "results/tflogs_flower_dataset/" --port 6000

In [ ]:
# models = ["ResNet50", "VGG16", "InceptionResnetV2", "Mobilenet"]
# batch_size = [32, 64, 1024]
# loss_func = [ "categorical_cross_entropy", ]
# optimizer = ["adam", ]
# learning_rate = 0.001

# for mod in models:
#     for loss_ in loss_func:
#         for op in optimizer:
#             for bs in batch_size:
#                 print(dict(model=mod, loss_func = loss_, optimizer=op, batch_sieze=bs, lr=learning_rate))